In [1]:
epochs = 100

In [2]:
%matplotlib inline
import os, sys

import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model
from sklearn.metrics import f1_score, classification_report, confusion_matrix, make_scorer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Input, Dense
from keras.models import Model
from keras.optimizers import SGD
from keras import regularizers
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from keras import backend as K
import tensorflow as tf

Using TensorFlow backend.
/home/admin/miniconda3/envs/da03/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/admin/miniconda3/envs/da03/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/admin/miniconda3/envs/da03/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/admin/miniconda3/env

In [3]:
sys.path.append('/home/admin/github/wordroid.sblo.jp/lib')
#from gkernel import GaussianKernel, GaussianKernel2, GaussianKernel3
from keras_ex.gkernel import GaussianKernel, GaussianKernel2, GaussianKernel3
from keras_ex.gkernel.sklearn import (
    RBFRegressor, RBFClassifier,
    make_model_gkernel1,
    make_model_gkernel2,
    make_model_gkernel3,
    make_model_out,
    make_model
)

In [4]:
# from sklearn.datasets import load_digits
# digits = load_digits()
# X, y = digits.data, digits.target
# X = X / 16
# X.shape, X.min(), X.max()

In [5]:
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original')
mnist

/home/admin/miniconda3/envs/da03/lib/python3.6/site-packages/sklearn/utils/deprecation.py:85: DeprecationWarning: Function fetch_mldata is deprecated; fetch_mldata was deprecated in version 0.20 and will be removed in version 0.22. Please use fetch_openml.
  warnings.warn(msg, category=DeprecationWarning)
/home/admin/miniconda3/envs/da03/lib/python3.6/site-packages/sklearn/utils/deprecation.py:85: DeprecationWarning: Function mldata_filename is deprecated; mldata_filename was deprecated in version 0.20 and will be removed in version 0.22. Please use fetch_openml.
  warnings.warn(msg, category=DeprecationWarning)


{'DESCR': 'mldata.org dataset: mnist-original',
 'COL_NAMES': ['label', 'data'],
 'target': array([0., 0., 0., ..., 9., 9., 9.]),
 'data': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)}

In [6]:
X, y = mnist.data, mnist.target
X = X / 255
X.shape, X.min(), X.max()

((70000, 784), 0.0, 1.0)

In [7]:
y.shape

(70000,)

In [8]:
y_cat = to_categorical(y, num_classes=10)
y_cat.shape

(70000, 10)

In [9]:
(
    x_train, x_test,
    y_cat_train, y_cat_test,
    y_train, y_test
) = train_test_split(X, y_cat, y,
                     test_size=0.4,
                     stratify=y, random_state=10002)

(
    x_train.shape, x_test.shape,
    y_cat_train.shape, y_cat_test.shape,
    y_train.shape, y_test.shape
)

((42000, 784), (28000, 784), (42000, 10), (28000, 10), (42000,), (28000,))

In [10]:
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV, validation_curve
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression, Lasso
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, roc_auc_score, r2_score, make_scorer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
import joblib

from lightgbm import LGBMClassifier

In [11]:
import warnings
warnings.filterwarnings('ignore')

In [12]:
def f1_scorer(estimator, X, y):
    pred = estimator.predict(X)
    s = f1_score(y, pred, average='macro')
    return s

## SVM

In [13]:
from sklearn.svm import SVC
SVC

sklearn.svm.classes.SVC

In [14]:
svc = SVC(gamma='scale')

In [15]:
%%time
svc.fit(x_train, y_train)

CPU times: user 3min 33s, sys: 367 ms, total: 3min 33s
Wall time: 3min 33s


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [16]:
%%time
'''taking long time for predict'''
pred = svc.predict(x_train)
pred.shape

CPU times: user 6min 8s, sys: 3.99 ms, total: 6min 8s
Wall time: 6min 8s


(42000,)

In [17]:
pred

array([2., 9., 9., ..., 8., 4., 1.])

In [18]:
print('F1_SCORE :', f1_score(y_train, pred, average='macro'))
print(classification_report(y_train, pred))
print(confusion_matrix(y_train, pred))

F1_SCORE : 0.9888827664346544
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99      4142
         1.0       0.99      0.99      0.99      4726
         2.0       0.99      0.99      0.99      4194
         3.0       0.99      0.99      0.99      4285
         4.0       0.99      0.99      0.99      4094
         5.0       0.99      0.99      0.99      3788
         6.0       0.99      0.99      0.99      4125
         7.0       0.98      0.99      0.99      4376
         8.0       0.99      0.99      0.99      4095
         9.0       0.99      0.98      0.98      4175

    accuracy                           0.99     42000
   macro avg       0.99      0.99      0.99     42000
weighted avg       0.99      0.99      0.99     42000

[[4121    1    2    1    3    2    3    2    4    3]
 [   0 4690   15    1    1    0    2    8    4    5]
 [   4    2 4155    4    6    0    1   15    6    1]
 [   0    4    8 4227    0   10    0   15   15    6]

In [19]:
%%time
'''taking long time for predict'''
pred_test = svc.predict(x_test)
pred_test.shape

CPU times: user 4min 5s, sys: 2 ms, total: 4min 5s
Wall time: 4min 5s


(28000,)

In [20]:
print('F1_SCORE :', f1_score(y_test, pred_test, average='macro'))
print(classification_report(y_test, pred_test))
print(confusion_matrix(y_test, pred_test))

F1_SCORE : 0.9767497984356794
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99      2761
         1.0       0.99      0.99      0.99      3151
         2.0       0.97      0.98      0.98      2796
         3.0       0.98      0.96      0.97      2856
         4.0       0.97      0.98      0.97      2730
         5.0       0.97      0.98      0.98      2525
         6.0       0.98      0.99      0.99      2751
         7.0       0.98      0.97      0.97      2917
         8.0       0.97      0.97      0.97      2730
         9.0       0.97      0.96      0.96      2783

    accuracy                           0.98     28000
   macro avg       0.98      0.98      0.98     28000
weighted avg       0.98      0.98      0.98     28000

[[2739    1    3    0    0    2    7    0    7    2]
 [   1 3122    8    3    1    1    0    3    5    7]
 [   6    4 2735    6   13    0    5   10   15    2]
 [   1    3   23 2750    1   26    2   17   24    9]